<a href="https://colab.research.google.com/github/adwoaelegba/refinesummary/blob/main/KoiosModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Koios Model


# Installing libraries

In [ ]:
pip install torch transformers datasets accelerate peft bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 85.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 98.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
pip install torch_xla[tpu] -f https://storage.googleapis.com/libtpu-releases/index.html


Looking in links: https://storage.googleapis.com/libtpu-releases/index.html


In [ ]:
pip install bs4

In [ ]:
pip install sentence_transformers

In [ ]:
pip install nltk

# Extraction Function
This function is meant to scrape the HTML from the site and feed it into the model

In [ ]:
def extraction_function(html):
    from bs4 import BeautifulSoup

    with open(html, "r", encoding="utf-8") as file:
        content = file.read()
        soup = BeautifulSoup(content, "html.parser")

    sections = {}
    current_heading = None
    current_text = []  # Initialize

    for elem in soup.find_all(["h1", "h2", "h3", "p"]):

        if elem.name in ["h1", "h2", "h3"]:
            if current_heading:
                sections[current_heading] = " ".join(current_text).strip()
            current_heading = elem.get_text().strip()
            current_text = []
        else:
            current_text.append(elem.get_text().strip())

    if current_heading and current_text:
        sections[current_heading] = " ".join(current_text).strip()


    return sections

Testing the extraction function

In [ ]:
html_file= "/content/drive/MyDrive/corpus/htmltest/www8_hp.html"
policy=extraction_function(html_file)
for section, text in policy.items():
    print(f"## {section} ###\n{text}\n")

## (No Subtitle) ###
HP respects your privacy

HP and its subsidiaries respect your privacy. This Privacy Statement informs you of our privacy practices and of the choices you can make about the way information about you, including from your online activity is collected and how that information is used by HP. This statement is readily available on our HP.com home page and at the bottom of every HP web page.

HP was a founding sponsor of the Council of Better Business Bureau's Online Privacy Program and is a BBB Accredited Business. HP privacy practices are consistent with the BBB Code of Business Practices and we proudly display the BBB seal.

Privacy BBB OnLinenon-HP site

In the development of HP's privacy policies and standards, we respect and take into account the major principles and frameworks around the world, including the OECD Guidelines on the Protection of Privacy and Transborder Flows, EU Directive 95/46/EC, APEC Privacy Framework, and the Madrid Resolution on International

# Extractive Summarization
This uses TD-IF to select the sentences with the highest importance. The measurement for importance is based on the privacy terms

In [ ]:
import torch
import nltk
from nltk.tokenize import sent_tokenize
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
sent_model=SentenceTransformer("all-MiniLM-L6-v2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
#Key privacy terms to prioritize
privacy_keywords={
    "privacy","data","personal","information","collection","third-party","cookies","retention",
    "cookies","sharing","consent","security","policy","tracking", "advertising","partners",
    "compliance","regulation","opt-out", "security"
}

In [ ]:
#Legal phrases that should be kept
legal_phrases={
    "third-party sharing", "right to be forgotten","legitimate interest","data retention",
    "opt-out option","cookie tracking"
}

In [ ]:
#Stopwords to remove to improve TD-IDF scores but keep the ones that are needed for legal phrases (i.e to be)
stopwords=list(set(nltk.corpus.stopwords.words("english")).union({"shall","hereby","thereof","whereas"}) - set("to be".split()))

In [ ]:
#Function to preprocess text, keeping legal phrases. Changed text to a single token
def preprocess_text(text):
    for phrase in legal_phrases:
        text=text.replace(phrase,phrase.replace(" ","_"))
    return text


In [ ]:
def td_extract_summary(text, boost_factor=2.0):
    text = preprocess_text(text)
    sentences = sent_tokenize(text)
    total_sentences = len(sentences)

    if total_sentences < 5:
        return text
    if total_sentences <= 10:
        num_sentences = max(5, int(total_sentences * 0.80))
    elif total_sentences <= 20:
        num_sentences = max(9, int(total_sentences * 0.5))
    else:
        num_sentences = max(11, int(total_sentences * 0.4))

    # TF-IDF
    vectorizer = TfidfVectorizer(stop_words=stopwords)
    tdidf_matrix = vectorizer.fit_transform(sentences)
    tdidf_scores = np.asarray(tdidf_matrix.mean(axis=0)).flatten()
    word_to_tdidf = dict(zip(vectorizer.get_feature_names_out(), tdidf_scores))

    sentence_weights = []
    for sentence in sentences:
        words = sentence.lower().split()
        weight = sum(word_to_tdidf.get(word, 0) for word in words)
        if any(word in privacy_keywords for word in words):
            weight *= boost_factor
        sentence_weights.append(weight)

    sentence_weights = np.array(sentence_weights)
    max_weight = np.max(sentence_weights) if np.max(sentence_weights) > 0 else 1
    sentence_weights = (sentence_weights / max_weight) * boost_factor

    # Encode sentences
    sentence_embeddings = sent_model.encode(sentences, convert_to_tensor=True)
    doc_embedding = torch.mean(sentence_embeddings, dim=0)

    # Compute similarities
    similarities = torch.nn.functional.cosine_similarity(sentence_embeddings, doc_embedding.unsqueeze(0))

    # Move sentence_weights to the same device as similarities
    sentence_weights_tensor = torch.tensor(sentence_weights, dtype=torch.float32, device=similarities.device)
    similarities = similarities * sentence_weights_tensor

    # Select top-ranked sentences
    top_indices = torch.topk(similarities, num_sentences).indices.cpu().numpy()

    # Maintain original order of selected sentences
    top_sentences = [sentences[i].replace("_", " ") for i in sorted(top_indices)]  # Restore legal phrases

    return " ".join(top_sentences)


In [ ]:
policy="""
As a website, we are committed to safeguarding your privacy and being transparent about how we collect and share your data with third parties. To improve your user experience, we may share certain information with trusted analytics providers who help us understand how you interact with our website, allowing us to optimize performance. Additionally, we may work with advertising networks to display personalized ads based on your browsing history and interests. These third-party networks may use cookies or other tracking technologies to gather data independently from our site. We may also share data with payment processors when you make a purchase to ensure transactions are processed securely and efficiently. In some cases, we partner with social media platforms to provide features such as social sharing or login integrations. When you interact with these platforms, your information may be shared according to their privacy policies. We also share data with service providers that assist us in delivering our services, including customer support or marketing communications. While we strive to carefully select third parties that comply with data protection laws, we cannot control how they use your data once shared. Your personal data may also be shared with law enforcement or regulatory authorities if required by law or to protect our legal rights. We ensure that any third party with whom we share data adheres to applicable data protection regulations such as the General Data Protection Regulation (GDPR) and California Consumer Privacy Act (CCPA). You have the ability to manage your data-sharing preferences by adjusting your privacy settings or using opt-out options available on our site. This includes controlling how third parties use your data for advertising purposes. We encourage you to review the privacy policies of any third-party services we work with to understand their data practices. By using our website, you consent to the sharing of your data with third parties as described in this policy. However, if you have any concerns or questions about our data-sharing practices, please feel free to contact us. Your privacy is important to us, and we strive to ensure that your data is handled responsibly.
"""

In [ ]:
#14 sentence paragraph to test on
policy="""
As a website, we are committed to safeguarding your privacy and being transparent about how we collect and share your data with third parties. To improve your user experience, we may share certain information with trusted analytics providers who help us understand how you interact with our website, allowing us to optimize performance. Additionally, we may work with advertising networks to display personalized ads based on your browsing history and interests. These third-party networks may use cookies or other tracking technologies to gather data independently from our site. We may also share data with payment processors when you make a purchase to ensure transactions are processed securely and efficiently. In some cases, we partner with social media platforms to provide features such as social sharing or login integrations. When you interact with these platforms, your information may be shared according to their privacy policies. We also share data with service providers that assist us in delivering our services, including customer support or marketing communications. While we strive to carefully select third parties that comply with data protection laws, we cannot control how they use your data once shared. Your personal data may also be shared with law enforcement or regulatory authorities if required by law or to protect our legal rights. We ensure that any third party with whom we share data adheres to applicable data protection regulations such as the General Data Protection Regulation (GDPR) and California Consumer Privacy Act (CCPA). You have the ability to manage your data-sharing preferences by adjusting your privacy settings or using opt-out options available on our site. This includes controlling how third parties use your data for advertising purposes. We encourage you to review the privacy policies of any third-party services we work with to understand their data practices. By using our website, you consent to the sharing of your data with third parties as described in this policy. However, if you have any concerns or questions about our data-sharing practices, please feel free to contact us. Your privacy is important to us, and we strive to ensure that your data is handled responsibly.
"""

summary=td_extract_summary(policy)
print("Extractive summary:", summary)

Extractive summary: 
As a website, we are committed to safeguarding your privacy and being transparent about how we collect and share your data with third parties. To improve your user experience, we may share certain information with trusted analytics providers who help us understand how you interact with our website, allowing us to optimize performance. We may also share data with payment processors when you make a purchase to ensure transactions are processed securely and efficiently. While we strive to carefully select third parties that comply with data protection laws, we cannot control how they use your data once shared. Your personal data may also be shared with law enforcement or regulatory authorities if required by law or to protect our legal rights. We ensure that any third party with whom we share data adheres to applicable data protection regulations such as the General Data Protection Regulation (GDPR) and California Consumer Privacy Act (CCPA). We encourage you to revie

# Abstractive Summarization

In [ ]:
import torch
from transformers import BartForConditionalGeneration, BartTokenizer, TrainingArguments, Trainer
from datasets import load_dataset
from peft import get_peft_model, LoraConfig, TaskType


In [ ]:
model_type="facebook/bart-large-cnn"
tokenizer=BartTokenizer.from_pretrained(model_type)
model=BartForConditionalGeneration.from_pretrained(model_type)

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [ ]:
#Apply LoRA
lora_config= LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    r=12,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none"
)

In [ ]:
model = get_peft_model(model, lora_config)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
privacy_policies = "/content/drive/MyDrive/corpus/training/privacy_final_policies.jsonl"


In [ ]:
import json

file_path = "/content/drive/MyDrive/corpus/training/privacy_final_policies.jsonl"

# Read the JSONL file
data = []
with open(file_path, "r", encoding="utf-8") as f:
    for line in f:
        data.append(json.loads(line))  # Load each line as a dictionary

# Print the first few examples
for i, entry in enumerate(data[:3]):
    print(f"Example {i+1}:")
    print("Input:", entry["input"])
    print("Output:", entry["output"])
    print("="*50)


Example 1:
Input: <section>
<h2 />
<p>Tuesday 01 November 2011 inShare PRINT A A A Before using the Website (that is, any site operated by Independent Digital News and Media Limited) it is your responsibility to read the Privacy Policy. You can notify us in the manner set out below if you do not want us to share information about you with third parties. Independent Digital News and Media Limited is hereinafter referred to in this Privacy Policy as IDNM. Please note that IDNM may revise this Privacy Policy at any time by posting an update on its website. Please ensure that you check this website from time to time to review the current privacy policy against any preferences you may have indicated. This Privacy Policy was last updated on the date indicated above. * Code of Conduct * Terms & Policies * Advertising terms and conditions * Competition terms and conditions * External contributors: Terms & conditions for freelancers, bloggers and contributors of user-generated content * Work ex

In [ ]:
dataset = load_dataset("json", data_files=privacy_policies, split="train")
print(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['input', 'output'],
    num_rows: 29
})


In [ ]:
print(dataset[0])  # Prints the first example


{'input': '<section>\n<h2 />\n<p>Tuesday 01 November 2011 inShare PRINT A A A Before using the Website (that is, any site operated by Independent Digital News and Media Limited) it is your responsibility to read the Privacy Policy. You can notify us in the manner set out below if you do not want us to share information about you with third parties. Independent Digital News and Media Limited is hereinafter referred to in this Privacy Policy as IDNM. Please note that IDNM may revise this Privacy Policy at any time by posting an update on its website. Please ensure that you check this website from time to time to review the current privacy policy against any preferences you may have indicated. This Privacy Policy was last updated on the date indicated above. * Code of Conduct * Terms & Policies * Advertising terms and conditions * Competition terms and conditions * External contributors: Terms & conditions for freelancers, bloggers and contributors of user-generated content * Work experie

In [ ]:
#Tokenization function (old but used in getting the tokenized dataset)
def tokenize(policies):
    inputs = tokenizer(policies["input"], max_length=1024, truncation=True)
    labels = tokenizer(policies["output"], max_length=256, truncation=True)
    inputs["labels"] = labels["input_ids"]
    return inputs

In [ ]:
print(model.config.max_position_embeddings)


1024


In [ ]:
#New tokenization function
def tokenization(policies):
    inputs = tokenizer(
        policies["input"],
        padding="max_length",
        truncation=True,
        max_length=1024
    )

    labels = tokenizer(
        policies["output"],
        padding="max_length",
        truncation=True,
        max_length=1024
    )

    return {
        "input_ids": inputs["input_ids"],
        "attention_mask": inputs["attention_mask"],
        "labels": labels["input_ids"]
    }



In [ ]:
tokenized_policies=dataset.map(tokenize, batched=True)

Map:   0%|          | 0/29 [00:00<?, ? examples/s]

In [ ]:
# Training arguments
train_args = TrainingArguments(
    output_dir="./bart_lora_privacy",
    evaluation_strategy="no",
    learning_rate=2e-4,
    per_device_train_batch_size=4,
    num_train_epochs=3,
    save_strategy="epoch",
    fp16=True,
    logging_dir="./logs"
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    pad_to_multiple_of=8
)


In [ ]:
model_trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=tokenized_policies,
    data_collator=data_collator
)

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "false"


In [ ]:
model_trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: naelegba (naelegba-ashesi-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


TrainOutput(global_step=24, training_loss=2.954771359761556, metrics={'train_runtime': 45.7605, 'train_samples_per_second': 1.901, 'train_steps_per_second': 0.524, 'total_flos': 142572366397440.0, 'train_loss': 2.954771359761556, 'epoch': 3.0})

In [ ]:
# Saving the fine-tuned model
model.save_pretrained("/content/drive/MyDrive/corpus/Koios Model")
tokenizer.save_pretrained("/content/drive/MyDrive/corpus/Koios Model")

('/content/drive/MyDrive/corpus/Koios Model/tokenizer_config.json',
 '/content/drive/MyDrive/corpus/Koios Model/special_tokens_map.json',
 '/content/drive/MyDrive/corpus/Koios Model/vocab.json',
 '/content/drive/MyDrive/corpus/Koios Model/merges.txt',
 '/content/drive/MyDrive/corpus/Koios Model/added_tokens.json')

# Final Integration Pipeline
This is the combination of the 3 separate functions that have been developed individually, **the extractive function for the model, extractive summarization model and finally, the abstractive model.** Now, this pipeline combines each of them and produces modular output(summary by section)

In [ ]:
#Load model
koios_model_path="/content/drive/MyDrive/corpus/Koios Model"
tokenizer=BartTokenizer.from_pretrained(koios_model_path)
koios_model=BartForConditionalGeneration.from_pretrained(koios_model_path)

In [ ]:
#Testing using a document in the dataset
url="/content/drive/MyDrive/corpus/htmltest/www8_hp.html"
extracted_summary=extraction_function(url)
final_summary={}


for current_heading, current_text in extracted_summary.items():
        extractive_mod_summary = td_extract_summary(current_text)
        if not isinstance(extractive_mod_summary, str):
           print(f"Error: extractive_mod_summary is {type(extractive_mod_summary)} instead of str.")
           print(f"Value: {extractive_mod_summary}")
        inputs = tokenizer(extractive_mod_summary, max_length=1024, truncation=True, padding="max_length", return_tensors="pt")
        summary_ids = koios_model.generate(**inputs, max_length=200, min_length=50, length_penalty=2.0)
        abstractive_mod_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        final_summary[current_heading] = abstractive_mod_summary
print("\n **Final Summarized Sections**")
for heading, summary in final_summary.items():
  print(f"\n **{heading}**\n{summary}")




 **Final Summarized Sections**

 **(No Subtitle)**
HP and its subsidiaries respect your privacy. This Privacy Statement informs you of the choices you can make about the way information about you, including from your online activity is collected and how that information is used by HP. In compliance with the Safe Harbor Principles and BCR requirements, HP commits to resolve complaints about your privacy and our collection or use of your personal information.

 **Scope of this Privacy Statement**
This Privacy Statement applies to all HP-owned websites, domains, services, applications, and products. A privacy policy or statement specific to a particular HP program, product, or service may supersede or supplement this Privacy Statement. We encourage you to review the privacy policy of any site you interact with before allowing the collection and use of your personal information.

 **Collection of personal information**
HP collects, exports, and uses personal information to manage your rel

# Feedback from 1st Koios Model
This model does the job of summarization, highlighting key aspects. However, from reading the original text, some crucial information is still missing. This next set of training works on the extractive side, focusing on increasing selection of sentences so it can pick more while still using the same abstractive model to compare.

# Trying the LED pretrained model

In [ ]:
import torch
from transformers import LEDTokenizer, LEDForConditionalGeneration
from datasets import load_dataset

In [ ]:
privacy_policies = "/content/drive/MyDrive/corpus/training/privacy_final_policies.jsonl"


In [ ]:
dataset = load_dataset("json", data_files=privacy_policies, split="train")
print(dataset)

Dataset({
    features: ['input', 'output'],
    num_rows: 29
})


In [ ]:
model_name="allenai/led-base-16384"
tokenizer=LEDTokenizer.from_pretrained("allenai/led-base-16384")
model=LEDForConditionalGeneration.from_pretrained(model_name)


In [ ]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

LEDForConditionalGeneration(
  (led): LEDModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): LEDEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): LEDLearnedPositionalEmbedding(16384, 768)
      (layers): ModuleList(
        (0-5): 6 x LEDEncoderLayer(
          (self_attn): LEDEncoderAttention(
            (longformer_self_attn): LEDEncoderSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (query_global): Linear(in_features=768, out_features=768, bias=True)
              (key_global): Linear(in_features=768, out_features=768, bias=True)
              (value_global): Linear(in_features=768, out_features=768, bias=True)
            )
            (output): Linear(in_features=768, out_features=768, bias=True)
          )
     

In [ ]:
#Apply LoRA
from peft import get_peft_model, LoraConfig, TaskType
lora_config= LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    r=12,
    target_modules=["q_proj", "v_proj"],
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none"
)

In [ ]:
model = get_peft_model(model, lora_config)

Preprocessing the Document

In [ ]:
def preprocess_text(text):
  inputs=tokenizer(text["input"],max_length=1024, truncation=True,padding="max_length")
  labels=tokenizer(text["output"],max_length=512, truncation=True,padding="max_length")
  inputs["labels"]=labels["input_ids"]
  return inputs

In [ ]:
dataset=dataset.map(preprocess_text,batched=True)
dataset.set_format(type="torch",columns=["input_ids","attention_mask", "labels"])

Map:   0%|          | 0/29 [00:00<?, ? examples/s]

In [ ]:
from transformers import Trainer, TrainingArguments
train_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/corpus/LEDTransformer",
    evaluation_strategy="no",
    learning_rate=1e-5,
    per_device_train_batch_size=1,
    num_train_epochs=15,
    save_strategy="epoch",
    fp16=False,
    logging_dir="./logs"
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
model_trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=dataset
)

In [ ]:
model_trainer.train()

Step,Training Loss


TrainOutput(global_step=435, training_loss=10.611869163074713, metrics={'train_runtime': 81.6027, 'train_samples_per_second': 5.331, 'train_steps_per_second': 5.331, 'total_flos': 294829566197760.0, 'train_loss': 10.611869163074713, 'epoch': 15.0})

In [ ]:
model.save_pretrained("/content/drive/MyDrive/corpus/LEDTransformer")
tokenizer.save_pretrained("/content/drive/MyDrive/corpus/LEDTransformer")

('/content/drive/MyDrive/corpus/LEDTransformer/tokenizer_config.json',
 '/content/drive/MyDrive/corpus/LEDTransformer/special_tokens_map.json',
 '/content/drive/MyDrive/corpus/LEDTransformer/vocab.json',
 '/content/drive/MyDrive/corpus/LEDTransformer/merges.txt',
 '/content/drive/MyDrive/corpus/LEDTransformer/added_tokens.json')

In [ ]:
#practice document
policy= """
Thank you for using our services. Your privacy is important to us. This Privacy Agreement outlines how we collect, use, and protect your information.

We use cookies to enhance your browsing experience. Cookies are small text files stored on your device that help us improve website functionality and personalize content. By using our website, you consent to the use of cookies, which may include essential cookies necessary for website operation, performance cookies that help us understand user behavior and improve site performance, functional cookies that enhance user experience by remembering preferences, and advertising cookies that provide targeted advertisements based on browsing history. You can manage your cookie preferences through your browser settings. Disabling cookies may affect website functionality.

We collect and process personal data to provide and improve our services. The types of data we collect include personal information such as name, email address, and contact details when provided voluntarily; usage data, which includes information about how you interact with our website, such as IP address, browser type, and pages visited; and transaction data if you make a purchase, including payment details necessary to process transactions. We use this data to provide, maintain, and improve our services, personalize user experience, ensure website security and prevent fraud, and comply with legal obligations. We do not sell or share your data with third parties without your consent, except when required by law.
"""

In [ ]:
policy2="""
A privacy agreement is a legal document that outlines how an organization collects, uses, stores, and protects user data. It serves as a contract between the company and its users, ensuring transparency in data handling practices. Privacy agreements typically cover details such as what types of personal information are collected, including names, email addresses, browsing history, and payment details. They also specify the purposes of data collection, whether for improving services, personalizing user experiences, or complying with legal obligations. Many privacy agreements include clauses about third-party data sharing, informing users whether their data will be shared with advertisers, analytics providers, or law enforcement. Additionally, they outline user rights, such as the ability to access, modify, or delete personal data, and often include instructions on how to exercise these rights. Security measures are another crucial aspect, detailing encryption methods, data retention policies, and protections against unauthorized access. With the rise of data breaches and cyber threats, privacy agreements help companies establish trust and demonstrate compliance with regulations like the General Data Protection Regulation (GDPR) and the California Consumer Privacy Act (CCPA). Companies must ensure that their privacy agreements are clear, easily accessible, and regularly updated to reflect changes in technology and legal requirements. Transparency is key, as vague or misleading agreements can result in legal consequences and damage to a company’s reputation. Users are encouraged to read privacy agreements carefully before consenting, as they provide insight into how their data is handled and whether they have control over it. Some companies offer opt-in or opt-out choices for data collection, giving users more control over their personal information. The complexity of privacy agreements can sometimes make them difficult to understand, which is why some organizations now provide simplified summaries alongside the full legal text. Businesses operating globally must account for differing privacy laws across jurisdictions, making compliance a continuous process. Privacy agreements also play a role in cookies and tracking technologies, informing users about their rights to manage cookie preferences. In some cases, companies may be required to obtain explicit consent before collecting certain types of data. As digital interactions continue to grow, privacy agreements will remain a fundamental aspect of online transactions and user trust. Organizations that prioritize privacy and data protection not only comply with legal standards but also foster stronger relationships with their users. Companies should also provide contact information for privacy-related inquiries, ensuring users have a way to address concerns or request clarifications. The evolving landscape of artificial intelligence and data analytics further underscores the importance of privacy agreements in safeguarding personal information. Ultimately, a well-crafted privacy agreement balances business needs with user rights, promoting ethical data practices in an increasingly digital world.
"""

# Testing LED Fine-Tuned Model

In [ ]:
import torch
from transformers import LEDTokenizer, LEDForConditionalGeneration

In [ ]:
LED_model_path="/content/drive/MyDrive/corpus/LEDTransformer"
tokenizer=LEDTokenizer.from_pretrained(LED_model_path)
LED_model=LEDForConditionalGeneration.from_pretrained(LED_model_path)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/648M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [ ]:
#extracting important sentences first so that it can extract that one instead of just choosing everything and truncating in the middle of a sentence
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

def extract_top_sentences(text, num_sentences=10):
    sentences = text.split(". ")  # Simple split on full stops(to indicate the end of a sentence)
    vectorizer = TfidfVectorizer()

    X = vectorizer.fit_transform(sentences)  # Converting to TF-IDF matrix
    td_scores = np.mean(X.toarray(), axis=1)  # Calculating the TF-IDF score per sentence

    top_sent_indices = np.argsort(td_scores)[-num_sentences:]  # Getting important sentences
    import_sentences = [sentences[i] for i in sorted(top_sent_indices)]

    return " ".join(import_sentences)


In [ ]:
#Splitting the text into chunks so to feed it to the model so it can properly analyse the sentences
def split_text(text, split_size=4096, overlap=512):
    words = text.split()  # Split into words to avoid cutting mid-sentence
    chunks = []

    for i in range(0, len(words), split_size - overlap):
        chunk = " ".join(words[i : i + split_size])  # Reassemble chunk
        chunks.append(chunk)

    return chunks


In [ ]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def sum_text(text):
    chunks = split_text(text)
    extracted_text = extract_top_sentences(" ".join(chunks))

    inputs = tokenizer(extracted_text, return_tensors="pt", padding=True, truncation=True, max_length=4096)
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)
    LED_model.to(device)

    summary_ids = LED_model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=600,
        min_length=200,
        length_penalty=1.0,
        num_beams=5,
        no_repeat_ngram_size=3
    )

    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)


In [ ]:
policy_sum=sum_text(policy2)
print("\nExtracted Summary:")
print(policy_sum)

Input ids are automatically padded from 284 to 1024 to be a multiple of `config.attention_window`: 1024



Extracted Summary:
Privacy agreements typically cover details such as what types of personal information are collected, including names, email addresses, browsing history, and payment details They also specify the purposes of data collection, whether for improving services, personalizing user experiences, or complying with legal obligations Many privacy agreements include clauses about third-party data sharing, informing users whether their data will be shared with advertisers, analytics providers, or law enforcement Additionally, they outline user rights, such as the ability to access, modify, or delete personal data, and often include instructions on how to exercise these rights With the rise of data breaches and cyber threats, privacy agreements help companies establish trust and demonstrate compliance with regulations like the General Data Protection Regulation (GDPR) and the California Consumer Privacy Act (CCPA) Companies must ensure that their privacy agreements are clear, easi

# Abstractive Model- Flan-T5 (Fine-Tuned)

In [ ]:
#importing
from datasets import load_dataset
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from peft import get_peft_model, LoraConfig, TaskType

In [ ]:
#Loading model
model_path="google/flan-t5-small"
tokenizer=T5Tokenizer.from_pretrained(model_path)
model=T5ForConditionalGeneration.from_pretrained(model_path)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 

In [ ]:
privacy_policies = "/content/drive/MyDrive/corpus/training/privacy_final_policies.jsonl"

In [ ]:
dataset = load_dataset("json", data_files=privacy_policies, split="train")
print(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['input', 'output'],
    num_rows: 29
})


In [ ]:
 #Apply LoRA
from peft import get_peft_model, LoraConfig, TaskType
lora_config= LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    r=12,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none"
)

In [ ]:
model=get_peft_model(model, lora_config)

In [ ]:
#tokenizer function, specifying that emojis should be added and that it should be in a fun way so that it matches the preferred output
def tokenizer_new (policies):
  inputs=["summarize with emojis: "+ text for text in policies["input"]]
  summaries=policies["output"]
  model_inputs=tokenizer(inputs,max_length=1024, truncation=True,padding="max_length")
  labels=tokenizer(summaries,max_length=512, truncation=True,padding="max_length")
  model_inputs["labels"]=labels["input_ids"]
  return model_inputs


In [ ]:
dataset=dataset.map(tokenizer_new,batched=True)

Map:   0%|          | 0/29 [00:00<?, ? examples/s]

Fine-tuning Flan T5

In [ ]:
train_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/corpus/Flan-T5",
    evaluation_strategy="no",
    learning_rate=1e-5,
    gradient_accumulation_steps=8,
    weight_decay=0.01,
    per_device_train_batch_size=2,
    num_train_epochs=9,
    save_strategy="epoch",
    fp16=True,
    logging_dir="./logs"
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
model_trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=dataset
)

In [ ]:
model_trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: naelegba (naelegba-ashesi-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss


TrainOutput(global_step=9, training_loss=0.0, metrics={'train_runtime': 55.2696, 'train_samples_per_second': 4.722, 'train_steps_per_second': 0.163, 'total_flos': 83989226323968.0, 'train_loss': 0.0, 'epoch': 8.533333333333333})

In [ ]:
model.save_pretrained("/content/drive/MyDrive/corpus/Flan-T5")
tokenizer.save_pretrained("/content/drive/MyDrive/corpus/Flan-T5")

('/content/drive/MyDrive/corpus/Flan-T5/tokenizer_config.json',
 '/content/drive/MyDrive/corpus/Flan-T5/special_tokens_map.json',
 '/content/drive/MyDrive/corpus/Flan-T5/spiece.model',
 '/content/drive/MyDrive/corpus/Flan-T5/added_tokens.json')

# Saving Fine-Tuned Model

In [ ]:
flan_model=T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/corpus/Flan-T5")
flan_tokenizer=T5Tokenizer.from_pretrained("/content/drive/MyDrive/corpus/Flan-T5")

# Testing The Fine-Tuned Model on Practice Paragraphs

In [ ]:
def summarize_final_text(text):

  input_policy=prompt = "Summarize the key points of this privacy policy in simple terms: "+ text
  inputs=flan_tokenizer(input_policy, return_tensors="pt", max_length=512, truncation=True)
  summary_ids=flan_model.generate(inputs["input_ids"], max_length=128,min_length=50,length_penalty=1.0,num_beams=4,do_sample=True,
    top_k=40,top_p=0.7,repetition_penalty=1.8)
  return flan_tokenizer.decode(summary_ids[0],skip_special_tokens=True)

In [ ]:
print(summarize_final_text(policy))

Understand how we use your personal information. Understand how we use your personal information. Understand how we use your personal data. Learn how we use your personal data to provide you with services and products that you may be able to purchase. Understand how we use your personal data to provide you with services and products that you may be able to purchase.


In [ ]:
def summarize_final_text1(text):

  #input_policy=prompt = "Summarize the key points of this privacy policy in simple terms: "+ text
  inputs=flan_tokenizer(text, return_tensors="pt", max_length=512, truncation=True)
  summary_ids=flan_model.generate(inputs["input_ids"], max_length=128,min_length=50,length_penalty=1.0,num_beams=4,do_sample=True,
  return flan_tokenizer.decode(summary_ids[0],skip_special_tokens=True)

In [ ]:
print(summarize_final_text1(policy))

We do not sell or share your personal data with third parties without your consent, except when required by law (including when required by law.) Except as required by law, we do not sell or share your data with third parties without your consent, except when required by law.


# Visual Feedback From Fine-Tuned Flan-T5 Small (Without ROUGE and BLEU testing)
The performance on the first practice paragraph was subpar. It summarised but it didn't use any articles or form any full sentences. It also does not include information that consumers may want to know

The performance of the model with different parameters in summarize_final_text1 is also subpar. The model only took one sentence out of the entire thing and repated it

# Trying Flan T5 Large without Fine-Tuning

In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments

In [ ]:
#Loading model
model_path="google/flan-t5-large"
tokenizer=T5Tokenizer.from_pretrained(model_path)
model=T5ForConditionalGeneration.from_pretrained(model_path)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
def summarize_final_text(text):
  inputs=tokenizer(text, return_tensors="pt", max_length=512, truncation=True)
  summary_ids=model.generate(inputs["input_ids"], max_length=200,min_length=50,length_penalty=1.0,num_beams=4,do_sample=True,  # Allow randomness
    top_k=40,top_p=0.7,repetition_penalty=1.8)
  return tokenizer.decode(summary_ids[0],skip_special_tokens=True)

In [ ]:
#Save model
save_directory = "/content/drive/MyDrive/corpus/Flan-T5-large"

# Save model and tokenizer
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

generation_params = {
    "max_length": 200,
    "min_length": 50,
    "length_penalty": 1.0,
    "num_beams": 4,
    "do_sample": True
}
torch.save(generation_params, f"{save_directory}/generation_params.pth")

print("Model, tokenizer, and generation parameters saved!")

Model, tokenizer, and generation parameters saved!


In [ ]:
#practice document
policy= """
Thank you for using our services. Your privacy is important to us. This Privacy Agreement outlines how we collect, use, and protect your information.

We use cookies to enhance your browsing experience. Cookies are small text files stored on your device that help us improve website functionality and personalize content. By using our website, you consent to the use of cookies, which may include essential cookies necessary for website operation, performance cookies that help us understand user behavior and improve site performance, functional cookies that enhance user experience by remembering preferences, and advertising cookies that provide targeted advertisements based on browsing history. You can manage your cookie preferences through your browser settings. Disabling cookies may affect website functionality.

We collect and process personal data to provide and improve our services. The types of data we collect include personal information such as name, email address, and contact details when provided voluntarily; usage data, which includes information about how you interact with our website, such as IP address, browser type, and pages visited; and transaction data if you make a purchase, including payment details necessary to process transactions. We use this data to provide, maintain, and improve our services, personalize user experience, ensure website security and prevent fraud, and comply with legal obligations. We do not sell or share your data with third parties without your consent, except when required by law.
"""

In [ ]:
print(summarize_final_text(policy))

This Privacy Agreement outlines how we collect, use, and protect your information. Cookies are small text files stored on your device that help us improve website functionality and personalize content. By using our website, you consent to the use of cookies. We collect and process personal data to provide and improve our services. The types of data we collect include personal information such as name, email address, and contact details when provided voluntarily; usage data, which includes information about how you interact with our website, such as IP address, browser type, and pages visited; and transaction data if you make a purchase, including payment details necessary to process transactions. We do not sell or share your data with third parties without your consent, except when required by law.


In [ ]:
new_policy="""
At Pinterest, we collect information to improve our services, enhance user experience, and ensure security. When you visit our website or use our services, we automatically gather data such as your IP address, browser type, device information, and browsing behavior through cookies and similar tracking technologies. If you create an account or make a purchase, we collect personal details like your name, email address, payment information, and shipping details. Additionally, when you interact with our customer support or provide feedback, we store your communications to assist you better. We may also collect data from third-party sources, such as social media platforms or marketing partners, to personalize your experience and provide relevant offers. Usage data, including how you navigate our website and which features you use, helps us refine our products and tailor content to your preferences. If you subscribe to our newsletter or promotional emails, we track engagement metrics like open rates and link clicks to optimize our communication. We ensure that all data collection follows legal guidelines and industry standards, and we give users control over their privacy settings. You can manage cookie preferences, update personal information, or opt out of marketing communications at any time. For security purposes, we monitor login activity and may use automated tools to detect fraud or suspicious behavior. Our data collection practices help us provide seamless service, troubleshoot technical issues, and comply with regulatory requirements. We also anonymize and aggregate certain data for analytical purposes, ensuring that individual identities remain protected. Your trust is important to us, and we are committed to transparency in how we collect, store, and use your information. By using our services, you consent to our data collection practices, which are detailed in our Privacy Policy. If you have any questions or concerns about how we handle your data, you can contact our support team or review our privacy settings.
"""

In [ ]:
print(summarize_final_text(new_policy))

We collect information to improve our services, enhance user experience, and ensure security. We may also collect data from third-party sources, such as social media platforms or marketing partners, to personalize your experience and provide relevant offers. Usage data, including how you navigate our website and which features you use, helps us refine our products and tailor content to your preferences. You can manage cookie preferences, update personal information, or opt out of marketing communications at any time. For security purposes, we monitor login activity and may use automated tools to detect fraud or suspicious behavior. Our data collection practices help us provide seamless service, troubleshoot technical issues, and comply with regulatory requirements. Your trust is important to us, and we are committed to transparency in how we collect, store, and use your information. By using our services, you consent to our data collection practices, which are detailed in our Privacy P

# Visual Assessment of Summaries From Flan T5- Large Without Fine Tuning
This model with a certain number of parameters seems to work better. Even though it does more of an extraction than rewording text

# Second Koios Final Pipeline (LED+ Flan T5 Large)

In [ ]:
#Practice document
url="/content/drive/MyDrive/corpus/htmltest/www8_hp.html"


In [ ]:
#importing libraries
from transformers import LEDTokenizer, LEDForConditionalGeneration
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

In [ ]:
#Loading LED model
LED_path="/content/drive/MyDrive/corpus/LEDTransformer"
LED_model=LEDForConditionalGeneration.from_pretrained(LED_path)
LED_tokenizer=LEDTokenizer.from_pretrained(LED_path)

In [ ]:
t5_path="/content/drive/MyDrive/corpus/Flan-T5-large"
t5_model=T5ForConditionalGeneration.from_pretrained(t5_path)
t5_tokenizer=T5Tokenizer.from_pretrained(t5_path)

In [ ]:
def extraction_function_text(html):
    from bs4 import BeautifulSoup

    with open(html, "r", encoding="utf-8") as file:
        content = file.read()
        soup = BeautifulSoup(content, "html.parser")

    sections = {}
    current_heading = None
    current_text = []

    for elem in soup.find_all(["h1", "h2", "h3", "p"]):

        if elem.name in ["h1", "h2", "h3"]:
            if current_heading:
                sections[current_heading] = " ".join(current_text).strip()
            current_heading = elem.get_text().strip()
            current_text = []
        else:
            current_text.append(elem.get_text().strip())

    if current_heading and current_text:
        sections[current_heading] = " ".join(current_text).strip()


    return sections

In [ ]:
#abstractive function
def abstract_final_text(text):


  inputs=t5_tokenizer(text, return_tensors="pt", max_length=512, truncation=True)
  summary_ids=t5_model.generate(inputs["input_ids"], max_length=200,min_length=50,length_penalty=1.0,num_beams=4,do_sample=True,
    top_k=40,top_p=0.7,repetition_penalty=1.8)
  return t5_tokenizer.decode(summary_ids[0],skip_special_tokens=True)

In [ ]:
#extractive function
def extract_final_text(text):
    chunks = split_text(text)
    extracted_text = extract_top_sentences(" ".join(chunks))

    inputs = LED_tokenizer(extracted_text, return_tensors="pt", padding=True, truncation=True, max_length=4096)
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]


    summary_ids = LED_model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=600,
        min_length=200,
        length_penalty=1.0,
        num_beams=5,
        no_repeat_ngram_size=3
    )

    return LED_tokenizer.decode(summary_ids[0], skip_special_tokens=True)


In [ ]:
import torch

def koios_pipeline2(url, LED_model, LED_tokenizer, t5_model, t5_tokenizer):

    # Extract text from HTML
    extracted_summary = extraction_function(url)

    final_summary = {}

    # Loop through sections and summarize
    for current_heading, current_text in extracted_summary.items():

        # Ensure text is valid
        if not isinstance(current_text, str) or not current_text.strip():
            print(f"Skipping empty or invalid section: {current_heading}")
            continue

        try:
            #  Extractive Summarization using LED model
            inputs = LED_tokenizer(current_text, max_length=4096, truncation=True, return_tensors="pt")
            summary_ids = LED_model.generate(**inputs, max_length=200, min_length=50, length_penalty=2.0)
            extractive_summary =LED_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

            # Ensure extractive summary is valid
            if not isinstance(extractive_summary, str) or not extractive_summary.strip():
                print(f"Error: extractive_summary is {type(extractive_summary)} instead of str for section {current_heading}")
                continue

            # Abstractive Summarization using T5 model
            inputs = t5_tokenizer(extractive_summary, max_length=1024, truncation=True, padding="max_length", return_tensors="pt")
            summary_ids = t5_model.generate(**inputs, max_length=200, min_length=50, length_penalty=2.0)
            abstractive_summary = t5_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

            # Store final summary
            final_summary[current_heading] = abstractive_summary

        except Exception as e:
            print(f"Error processing section '{current_heading}': {e}")

    return final_summary


In [ ]:
#new one using pre defined functions

def koios_pipeline2(url, LED_model, LED_tokenizer, t5_model, t5_tokenizer):


    extracted_summary = extraction_function(url)

    final_summary = {}


    for current_heading, current_text in extracted_summary.items():


        if not isinstance(current_text, str) or not current_text.strip():
            print(f"Skipping empty or invalid section: {current_heading}")
            continue

        try:
            # Extractive Summarization using LED model
            extractive_summary =extract_final_text(current_text)

            # Ensure extractive summary is valid
            if not isinstance(extractive_summary, str) or not extractive_summary.strip():
                print(f"Error: extractive_summary is {type(extractive_summary)} instead of str for section {current_heading}")
                continue

            #  Abstractive Summarization using T5 model
            abstractive_summary =abstract_final_text(extractive_summary)

            # Store final summary
            final_summary[current_heading] = abstractive_summary

        except Exception as e:
            print(f"Error processing section '{current_heading}': {e}")

    return final_summary


In [ ]:
final_summary = koios_pipeline2(url, LED_model, LED_tokenizer, t5_model, t5_tokenizer)

# Print the results
print("\n **Final Summarized Sections**")
for heading, summary in final_summary.items():
    print(f"\n **{heading}**\n{summary}")

Input ids are automatically padded from 491 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 281 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 680 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 212 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 419 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 53 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 397 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 144 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 219 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 72 to 1024 to be a multip


 **Final Summarized Sections**

 **(No Subtitle)**
Privacy BBB OnLinenon-HP site In the development of HP's privacy policies and standards, we respect and take into account the major principles and frameworks around the world, including the OECD Guidelines on the Protection of Privacy and Transborder Flows, EU Directive 95/46/EC, APEC Privacy Framework, and the Madrid Resolution on International Privacy Standards HP has certified that it adheres to the Safe Harbor Privacy Principles of notice, choice, onward transfer, security, data integrity, access, and enforcement HP has also established a set of binding corporate rules (“BCR”), which have been approved by the majority of Data Protection Regulators in the EEA and Switzerland, effective June 2011 In compliance with the Safe Harbor Principles and BCR requirements, HP commits to resolve complaints about your privacy and our collection or use of your personal information citizens or residents regarding transfers of their personal data 